In [6]:
import sys
import string
import os
import numpy as np
import pandas as pd
import astropy.units as U
import glob
import csv
sys.path.append(os.path.dirname(os.getcwd()))
import alma as ual

# Selecting csv path and output path directory
# def get_user_input():
#     csv_file_path = input("Inserisci il percorso completo del file CSV: ")
#     output_directory = input("Inserisci il percorso completo della directory di output: ")
#     return csv_file_path, output_directory

# Setting directory
csv_file_path = r"C:\Users\lsann\Desktop\GitHub\ALMASim\almasim\metadata\qso_metadata.csv"
output_directory = r"C:\Users\lsann\Desktop\GitHub\ALMASim\almasim"

# Calculating Parameters to run tclean with the help of ALMASim

db = pd.read_csv(csv_file_path)

def remove_non_numeric(text):
    """Rimuove i caratteri non numerici da una stringa."""
    numbers = "0123456789."
    return "".join(char for char in text if char in numbers)

def freq_supp_extractor(freq_sup, obs_freq):
    freq_band, n_channels, freq_mins, freq_maxs, freq_ds = [], [], [], [], []
    freq_sup = freq_sup.split('U')
    for i in range(len(freq_sup)):
        sup = freq_sup[i][1:-1].split(',')
        sup = [su.split('..') for su in sup][:2]
        freq_min, freq_max = float(remove_non_numeric(sup[0][0])), float(remove_non_numeric(sup[0][1]))
        freq_d = float(remove_non_numeric(sup[1][0]))
        freq_min = freq_min * U.GHz 
        freq_max = freq_max * U.GHz
        freq_d = freq_d * U.kHz
        freq_d = freq_d.to(U.GHz)
        freq_b = freq_max - freq_min
        n_chan = int(freq_b / freq_d)
        freq_band.append(freq_b)
        n_channels.append(n_chan)
        freq_mins.append(freq_min)
        freq_maxs.append(freq_max)
        freq_ds.append(freq_d)
    freq_ranges = np.array([[freq_mins[i].value, freq_maxs[i].value] for i in range(len(freq_mins))])
    idx_ = np.argwhere((obs_freq >= freq_ranges[:, 0]) & (obs_freq <= freq_ranges[:, 1]))[0][0]
    freq_range = freq_ranges[idx_]
    band_range = freq_range[1] - freq_range[0]
    n_channels = n_channels[idx_]
    central_freq = freq_range[0] + band_range / 2
    freq_d = freq_ds[idx_]
    return band_range * U.GHz, central_freq * U.GHz, n_channels, freq_d

def get_band_central_freq(band):
    band_central_freqs = {
        1: 38,
        2: 78.5,
        3: 100,
        4: 143,
        5: 217,
        6: 250,
        7: 353,
        8: 545,
        9: 650,
        10: 850
    }
    return band_central_freqs.get(band, None)

band_range, central_freq, n_channels, freq_q = zip(*db.apply(lambda row: freq_supp_extractor(row['Freq.sup.'], row['Freq']), axis=1))
spw = [f"{cf - br/2}~{cf + br/2}" for cf, br in zip(central_freq, band_range)]
spw_str = f"spw={spw}"
antenna_arrays = db['antenna_arrays']
max_baseline = db['antenna_arrays'].apply(lambda x: ual.get_max_baseline_from_antenna_array(antenna_array=x, master_path=output_directory))
band = db['Band']
central_freq = band.apply(get_band_central_freq)
data = {'central_freq': central_freq,
        'max_baseline': max_baseline}
df = pd.DataFrame(data)
beam_size = df.apply(lambda row: ual.estimate_alma_beam_size(row['central_freq'], row['max_baseline']), axis=1)
cell_size = beam_size.values / 5
cell_size = np.array(cell_size)

n_pix = np.floor(1.5 * db['FOV'].values * 3600 / cell_size)
n_pix_power_of_2 = np.power(2, np.ceil(np.log2(n_pix)))
alma_source_combined = db['ALMA_source_name'] + "_" + db['member_ous_uid'].str.replace(':', '_').str.replace('/', '_')
output_file_path = os.path.join(output_directory, 'metadata/calculated_params2.csv')
with open(output_file_path, 'w') as f:
    f.write('sb_name,ALMA_source_combined,cell_size,n_pixels,band_range,spw,nchan,beam_size\n')
    for i, sb_name in enumerate(db['SB_name']):
        f.write(f'{sb_name},')
        f.write(f"{alma_source_combined[i]},")
        f.write(f"{cell_size[i]},")  
        f.write(f"{n_pix_power_of_2[i]},") 
        f.write(f"{band_range[i]},")
        f.write(f"{spw[i]},")
        f.write(f"{n_channels[i]},")
        f.write(f"{beam_size[i]}\n")
print(n_channels)

(964, 478, 1656, 964, 964, 1665, 63, 964, 1656, 964, 964, 478, 478, 478, 969, 969, 964, 964, 964, 512, 964, 478, 964, 1656, 964, 63, 63, 63, 63, 63, 63)
